# Stablecoin Billionaires<br> Descriptive Analysis of the Ethereum-based Stablecoin ecosystem 

## by Anton Wahrstätter, 01.07.2020

## Script to prepare the data

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter

<center></center>

## Data

In [ ]:
#tether
tether_chunk_0 = 'data/tether/transfer/0_tether_transfer_4638568-8513778.csv'
tether_chunk_1 = 'data/tether/transfer/1_tether_transfer_8513799-8999999.csv'
tether_chunk_2 = 'data/tether/transfer/2_tether_transfer_9000000-9799999.csv'
tether_chunk_3 = 'data/tether/transfer/3_tether_transfer_9800000-10037842.csv'
tether_chunk_4 = 'data/tether/transfer/4_tether_transfer_10037843-10176690.csv'
tether_chunk_5 = 'data/tether/transfer/5_tether_transfer_10176691-10370273.csv' 
tether_chunk_0_1 = 'data/tether/transfer/0_tether_transfer_4638568-8999999.csv'
tether_transfer = 'data/tether/transfer/tether_transfers.csv'
tether_issue = 'data/tether/issue/tether_issue.csv'
tether_destroyedblackfunds = 'data/tether/destroyedblackfunds/tether_destroyedblackfunds.csv'
tether_tx_count_to = 'plots/tether/tether_tx_count_to.csv'
tether_tx_count_from = 'plots/tether/tether_tx_count_from.csv'

#usdc
usdc_transfer = 'data/usdc/transfer/0_usdc_transfer_6082465-10370273.csv' 
usdc_mint = 'data/usdc/mint/usdc_mint.csv' 
usdc_burn = 'data/usdc/burn/usdc_burn.csv' 
usdc_tx_count_to = 'plots/usdc/usdc_tx_count_to.csv'
usdc_tx_count_from = 'plots/usdc/usdc_tx_count_from.csv'

#paxos
paxos_transfer = 'data/paxos/transfer/0_paxos_transfer_6294931-10370273.csv' 
paxos_mint = 'data/paxos/supplyincreased/paxos_supplyincreased.csv'
paxos_burn = 'data/paxos/supplydecreased/paxos_supplydecreased.csv'
paxos_tx_count_to = 'plots/paxos/paxos_tx_count_to.csv'
paxos_tx_count_from = 'plots/paxos/paxos_tx_count_from.csv'

#dai
dai_transfer = 'data/dai/transfer/0_dai_transfer_8928158-10370273.csv' 
dai_mint = 'data/dai/mint/dai_mint.csv'
dai_burn = 'data/dai/burn/dai_burn.csv'
dai_tx_count_to = 'plots/dai/dai_tx_count_to.csv'
dai_tx_count_from = 'plots/dai/dai_tx_count_from.csv'

#trueusd
trueusd_transfer = 'data/trueusd/transfer/0_trueUSD_transfer_5198636-10370273.csv' 
trueusd_mint = 'data/trueusd/mint/trueusd_mint.csv'
trueusd_mint_old = 'data/trueusd/mint/trueusd_mint_old.csv'
trueusd_burn = 'data/trueusd/burn/trueusd_burn.csv'
trueusd_burn_old = 'data/trueusd/burn/trueusd_burn_old.csv'

#binanceusd
binanceusd_transfer = 'data/binanceusd/transfer/0_binanceusd_transfer_8493105-10370273.csv' 
binanceusd_mint = 'data/binanceusd/supplyincreased/binanceusd_supplyincreased.csv'
binanceusd_burn = 'data/binanceusd/supplydecreased/binanceusd_supplydecreased.csv'
binanceusd_tx_count_to = 'plots/binanceusd/binanceusd_tx_count_to.csv'
binanceusd_tx_count_from = 'plots/binanceusd/binanceusd_tx_count_from.csv'

#husd
husd_transfer = 'data/husd/transfer/0_husd_transfer_8174400-10370273.csv' 
husd_mint = 'data/husd/issue/husd_issue.csv'
husd_burn = 'data/husd/redeem/husd_redeem.csv'
husd_tx_count_to = 'plots/husd/husd_tx_count_to.csv'
husd_tx_count_from = 'plots/husd/husd_tx_count_from.csv'

## Concentrate datasets

In [ ]:
def concentrate_data():
    df = pd.concat([pd.read_csv(tether_chunk_0), 
                    pd.read_csv(tether_chunk_1), 
                    pd.read_csv(tether_chunk_2), 
                    pd.read_csv(tether_chunk_3), 
                    pd.read_csv(tether_chunk_4),
                    pd.read_csv(tether_chunk_5)], ignore_index=True)
    df.to_csv('data/tether/transfer/tether_transfers.csv', index=False)
    return   

<center></center>

<center></center>

# Prepare Transfer Data

## Balances

In [ ]:
#works great for up to 18 decimals
#needs much RAM
pd.options.mode.chained_assignment = None
def get_balances(_df, decimals):
    token = _df.split('/')[1]
    print("Start with {}".format(token))
    df = pd.read_csv(_df)
    froms = df[['txfrom', 'txvalue']]
    froms['txvalue'] = froms['txvalue'].apply(lambda x: int(x)*-1)
    tos = df[['txto', 'txvalue']]
    tos['txvalue'] = tos['txvalue'].apply(lambda x: int(x))
    outs = froms.groupby("txfrom").sum().reset_index().rename(columns={"txfrom":"txto"})
    ins = tos.groupby("txto").sum().reset_index()
    balance = outs.append(ins).groupby("txto").sum()
    balance = balance / 10**decimals
    balance = balance.reset_index().rename(columns={"txto":"address"}).set_index("address").sort_values('txvalue')
    balance.to_csv('plots/{}/{}_balances.csv'.format(token, token))
    
get_balances(tether_transfer, 6)    
get_balances(binanceusd_transfer, 18)
get_balances(husd_transfer, 8)
get_balances(dai_transfer, 18)
get_balances(trueusd_transfer, 18)
get_balances(usdc_transfer, 6)
get_balances(paxos_transfer, 18)
get_balances(sai_transfer, 18)

## Remove burned tokens from Tether balances

In [ ]:
df = pd.read_csv('plots/tether/tether_balances.csv', index_col='address')
burn = pd.read_csv(tether_destroyedblackfunds).loc[:,['address', 'txvalue']].set_index('address')
burn['txvalue'] = burn['txvalue'] /-10**6
_df = df.append(burn)
_df.loc['0xc6cde7c39eb2f0f0095f41570af89efc2c1ea828',:]=108850 # bitfinex multisig
_df = _df.groupby(_df.index).sum().sort_values('txvalue')
_df.to_csv('plots/tether/tether_balances.csv')

<center></center>

<center></center>

<center></center>

In [ ]:
# depreciated, but needs less RAM
# works well for small number decimal coins
def get_balances(dflist, decimals=0, chunked=False):
    counter = 0
    for chunk in dflist:
        token = chunk.split('/')[1]
        _chunk = pd.read_csv(chunk)
        froms = _chunk[['txfrom', 'txvalue']].set_index('txfrom')
        froms['txvalue'] = froms['txvalue'].apply(lambda x: int(x)/(10**decimals)*-1)
        tos =  _chunk[[ 'txto', 'txvalue']].set_index('txto')
        tos['txvalue'] = tos['txvalue'].apply(lambda x: int(x)/(10**decimals))
        df = tos.append(froms)
        df = df.groupby(df.index).sum()
        if chunked:
            df.to_csv('plots/{}/{}_balances_chunk_{}.csv'.format(token, token, counter))
        else:
            df.to_csv('plots/{}/{}_balances.csv'.format(token, token))
        counter += 1
    return
        
usdt = [tether_chunk_0, tether_chunk_1, tether_chunk_2, tether_chunk_3, tether_chunk_4, tether_chunk_5]
get_balances(usdt, chunked=True)
get_balances([usdc_transfer])
get_balances([paxos_transfer], 18)
get_balances([dai_transfer])
get_balances([trueusd_transfer], 18)
get_balances([husd_transfer])
get_balances([binanceusd_transfer], 18)

<center></center>

## Concentrate Chunks (Balances)

<center></center>

In [ ]:
aa = pd.read_csv('plots/tether/tether_balances_chunk_0.csv', index_col=0)
bb = pd.read_csv('plots/tether/tether_balances_chunk_1.csv', index_col=0)
cc = pd.read_csv('plots/tether/tether_balances_chunk_2.csv', index_col=0)
dd = pd.read_csv('plots/tether/tether_balances_chunk_3.csv', index_col=0)
ee = pd.read_csv('plots/tether/tether_balances_chunk_4.csv', index_col=0)
ff = pd.read_csv('plots/tether/tether_balances_chunk_5.csv', index_col=0)
df = aa.append([bb,cc,dd,ee,ff])
df = df.groupby(df.index).sum()
df = df.sort_values('txvalue')
df.to_csv('plots/tether/tether_balances.csv')

<center></center>

<center></center>

## Transfer Counter

In [ ]:
def from_to_tx_counter(dflist):
    from_count = Counter()
    to_count = Counter()
    for chunk in dflist:
        token = chunk.split('/')[1]
        df = pd.read_csv(chunk)
        froms = Counter(df['txfrom'])
        tos = Counter(df['txto'])
        from_count = from_count + froms
        to_count = to_count + tos

    df_from = pd.DataFrame(dict(from_count).values(), index=dict(from_count).keys()).rename(columns={0: 'txs'})
    df_to = pd.DataFrame(dict(to_count).values(), index=dict(to_count).keys()).rename(columns={0: 'txs'})
    df_from.to_csv('plots/{}/{}_tx_count_from.csv'.format(token, token))
    df_to.to_csv('plots/{}/{}_tx_count_to.csv'.format(token, token))
    return
    
    
usdt = [tether_chunk_0, tether_chunk_1, tether_chunk_2, tether_chunk_3, tether_chunk_4, tether_chunk_5]
from_to_tx_counter(usdt)
from_to_tx_counter([usdc_transfer])
from_to_tx_counter([paxos_transfer])
from_to_tx_counter([dai_transfer])
from_to_tx_counter([trueusd_transfer])
from_to_tx_counter([husd_transfer])
from_to_tx_counter([binanceusd_transfer])
from_to_tx_counter([sai_transfer])

<center></center>

<center></center>

# Create plot data

# Transfers over date

In [ ]:
def extract_data(df):
    dates = df.apply(lambda x: str(datetime.utcfromtimestamp(x))[:10])
    txs = list(Counter(dates).values())
    
    a = dates.iloc[0]
    b = dates.iloc[-1]
    idx = pd.date_range(a,b)
    
    df = pd.DataFrame(txs, index=np.unique(dates), columns=['txs'] )
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    
    da, tx = df.index.tolist(), df['txs'].tolist()
    return da, tx

def create_plot_txs_over_date(df):
    dates =  []
    txs = []
    token = df.split('/')[1]
    for chunk in pd.read_csv(df, chunksize=100000):
        da, tx = extract_data(chunk['timestamp'])
        dates = dates + da
        txs = txs + tx
    df = pd.DataFrame({'dates': dates, 'txs': txs}).groupby('dates', as_index=False).sum()
    df.to_csv('plots/{}/{}_txs_over_date.csv'.format(token, token))
    return
    
create_plot_txs_over_date(tether_transfer)
create_plot_txs_over_date(usdc_transfer)
create_plot_txs_over_date(paxos_transfer)
create_plot_txs_over_date(dai_transfer)
create_plot_txs_over_date(trueusd_transfer)
create_plot_txs_over_date(binanceusd_transfer)
create_plot_txs_over_date(husd_transfer)
create_plot_txs_over_date(sai_transfer)

<center></center>

<center></center>

# Transfers to new addresses

In [ ]:
tos = set()
def _add(x):
    global tos
    tos.add(x)
    return 1

def extract_uniques(df):
    dates = df['timestamp'].apply(lambda x: str(datetime.utcfromtimestamp(x))[:10])
    un = df['txto'].apply(lambda x: _add(x) if x not in tos else 0)
    
    a = dates.iloc[0]
    b = dates.iloc[-1]
    idx = pd.date_range(a,b)
    df = pd.DataFrame({'dates':dates, 'uniques':un})
    df = df.groupby('dates', as_index = False).sum()
    df = df.set_index('dates')
    
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    return df.index.tolist(), df['uniques'].tolist()

def create_plot_unique_recipients_over_date(df):
    global tos
    dates =  []
    txs = []
    for i in df:
        token = i.split('/')[1]
        for chunk in pd.read_csv(i, chunksize=1000000):
            da, tx = extract_uniques(chunk[['timestamp', 'txto']])
            dates = dates + da
            txs = txs + tx
        
    df = pd.DataFrame({'dates': dates, 'txs': txs}).groupby('dates', as_index=False).sum()
    df.to_csv('plots/{}/{}_unique_recipients_over_date.csv'.format(token, token))
    tos = set()
    return

    
df = [tether_chunk_0, tether_chunk_1, tether_chunk_2, tether_chunk_3, tether_chunk_4, tether_chunk_5]
create_plot_unique_recipients_over_date(df)

create_plot_unique_recipients_over_date([usdc_transfer])
create_plot_unique_recipients_over_date([paxos_transfer])
create_plot_unique_recipients_over_date([husd_transfer])
create_plot_unique_recipients_over_date([binanceusd_transfer])
create_plot_unique_recipients_over_date([trueusd_transfer])
create_plot_unique_recipients_over_date([dai_transfer])
create_plot_unique_recipients_over_date([sai_transfer])#

<center></center>

<center></center>

# Transfers from new addresses

In [ ]:
froms = set()
def _add(x):
    global froms
    froms.add(x)
    return 1

def extract_uniques(df):
    dates = df['timestamp'].apply(lambda x: str(datetime.utcfromtimestamp(x))[:10])
    un = df['txfrom'].apply(lambda x: _add(x) if x not in froms else 0)
    a = dates.iloc[0]
    b = dates.iloc[-1]
    idx = pd.date_range(a,b)
    df = pd.DataFrame({'dates':dates, 'uniques':un})
    df = df.groupby('dates', as_index = False).sum()
    df = df.set_index('dates')
    
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    return df.index.tolist(), df['uniques'].tolist()


def create_plot_unique_senders_over_date(df):
    dates =  []
    txs = []
    for i in df:
        token = i.split('/')[1]
        for chunk in pd.read_csv(i, chunksize=10000000):
            da, tx = extract_uniques(chunk[['timestamp', 'txfrom']])
            dates = dates + da
            txs = txs + tx
    
    df = pd.DataFrame({'dates': dates, 'txs': txs}).groupby('dates', as_index=False).sum()
    df.to_csv('plots/{}/{}_unique_senders_over_date.csv'.format(token, token))
    return
    
df = [tether_chunk_0, tether_chunk_1, tether_chunk_2, tether_chunk_3, tether_chunk_4, tether_chunk_5]
create_plot_unique_senders_over_date(df)

create_plot_unique_senders_over_date([usdc_transfer])
create_plot_unique_senders_over_date([paxos_transfer])
create_plot_unique_senders_over_date([husd_transfer])
create_plot_unique_senders_over_date([binanceusd_transfer])
create_plot_unique_senders_over_date([trueusd_transfer])
create_plot_unique_senders_over_date([dai_transfer])
create_plot_unique_senders_over_date([sai_transfer])

<center></center>

# Unique Transfers per day

## Unique Recipients

In [ ]:
#timestamp first transfer event of contract
ts_tether = 1511827200
ts_usdc = 1536537600
ts_paxos = 1536537600
ts_dai = 1573603200
ts_sai = 1513555200
ts_trueusd = 1520208000
ts_husd = 1563580800
ts_binanceusd = 1568073600


def get_unique_recipients_per_day(df, ts):
    unique = dict()
    counter = 0
    token = df.split('/')[1]
    df = pd.read_csv(df)[['timestamp', 'txto']]
    while ts + 86400*counter < 1593561600:
        timefrom = ts + 86400*counter
        timeto = ts + 86400*(counter+1)
        uniques = len(df[(df['timestamp'] >=timefrom) & (df['timestamp'] < timeto)]['txto'].unique())
        date = str(datetime.utcfromtimestamp(timefrom))[:10]
        if date in unique.keys():
            unique[date] += uniques
        else:
            unique[date] = uniques
        counter += 1
    
    _df = pd.DataFrame(unique.values(), index=unique.keys()).rename(columns={0:'txs'})
    _df.to_csv('plots/{}/{}_unique_recipients_per_day_over_date.csv'.format(token, token))


get_unique_recipients_per_day(tether_transfer, ts_tether)
get_unique_recipients_per_day(usdc_transfer, ts_usdc)
get_unique_recipients_per_day(paxos_transfer, ts_paxos)
get_unique_recipients_per_day(dai_transfer, ts_dai)
get_unique_recipients_per_day(sai_transfer, ts_sai)
get_unique_recipients_per_day(husd_transfer, ts_husd)
get_unique_recipients_per_day(trueusd_transfer, ts_trueusd)
get_unique_recipients_per_day(binanceusd_transfer, ts_binanceusd)

<center></center>

## Unique Senders

In [ ]:
#timestamp first transfer event of contract
ts_tether = 1511827200
ts_usdc = 1536537600
ts_paxos = 1536537600
ts_dai = 1573603200
ts_sai = 1513555200
ts_trueusd = 1520208000
ts_husd = 1563580800
ts_binanceusd = 1568073600


def get_unique_senders_per_day(df, ts):
    unique = dict()
    counter = 0
    token = df.split('/')[1]
    df = pd.read_csv(df)[['timestamp', 'txfrom']]
    while ts + 86400*counter < 1593561600:
        timefrom = ts + 86400*counter
        timeto = ts + 86400*(counter+1)
        uniques = len(df[(df['timestamp'] >=timefrom) & (df['timestamp'] < timeto)]['txfrom'].unique())
        date = str(datetime.utcfromtimestamp(timefrom))[:10]
        if date in unique.keys():
            unique[date] += uniques
        else:
            unique[date] = uniques
        counter += 1

    _df = pd.DataFrame(unique.values(), index=unique.keys()).rename(columns={0:'txs'})
    _df.to_csv('plots/{}/{}_unique_senders_per_day_over_date.csv'.format(token, token))
    
get_unique_senders_per_day(tether_transfer, ts_tether)
get_unique_senders_per_day(usdc_transfer, ts_usdc)
get_unique_senders_per_day(paxos_transfer, ts_paxos)
get_unique_senders_per_day(dai_transfer, ts_dai)
get_unique_senders_per_day(sai_transfer, ts_sai)
get_unique_senders_per_day(husd_transfer, ts_husd)
get_unique_senders_per_day(trueusd_transfer, ts_trueusd)
get_unique_senders_per_day(binanceusd_transfer, ts_binanceusd)

<center></center>

<center></center>

# Average transfer value

In [ ]:
def create_plot_avg_txvalue(df, token, decimals):   
    df = df[['timestamp', 'txvalue']]
    dates = df['timestamp'].apply(lambda x: str(datetime.utcfromtimestamp(x))[:10])
    txvalue = df['txvalue']
    df = pd.DataFrame({'dates': dates, 'txvalue': txvalue.astype(float)/(10**decimals)})
    a = dates.iloc[0]
    b = dates.iloc[-1]
    idx = pd.date_range(a,b)
    df = df.groupby('dates', as_index=False).mean()
    df = df.set_index('dates')
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    df.to_csv('plots/{}/{}_avg_value_over_date.csv'.format(token, token))
    return

<center></center>

# Average gas price

In [ ]:
def create_plot_avg_gas(df, token):
    df = df[['timestamp', 'gas_price', 'gas_used']]
    dates = df['timestamp'].apply(lambda x: str(datetime.utcfromtimestamp(x))[:10])
    df = pd.DataFrame({'dates': dates, 'gas': df['gas_price']*df['gas_used']/(10**18)})
    a = dates.iloc[0]
    b = dates.iloc[-1]
    idx = pd.date_range(a,b)
    df = df.groupby('dates', as_index=False).mean()
    df = df.set_index('dates')
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    df.to_csv('plots/{}/{}_avg_gas_over_date.csv'.format(token, token))
    return

## Run both

In [ ]:
for i in [(paxos_transfer, 18), (usdc_transfer, 6), (husd_transfer, 8), 
          (dai_transfer, 18), (sai_transfer, 18), (trueusd_transfer, 18), (binanceusd_transfer, 18)]:
    df = pd.read_csv(i[0])
    token = i[0].split('/')[1]
    create_plot_avg_txvalue(df, token, i[1])
    create_plot_avg_gas(df, token)

<center></center>

<center></center>

# Circulating supply 


## Prepare tokens without Mint/Burn Events (DAI)


In [ ]:
df = pd.read_csv(dai_transfer)

In [ ]:
mint = df[df['txfrom'] == '0x0000000000000000000000000000000000000000'].reset_index().drop('index', axis = 1).rename(columns={'txfrom': 'address'})
burn = df[df['txto'] == '0x0000000000000000000000000000000000000000'].reset_index().drop('index', axis = 1).rename(columns={'txfrom': 'address'})

mint.to_csv('data/dai/mint/dai_mint.csv')
burn.to_csv('data/dai/burn/dai_burn.csv')

## Create Plot for circulating supply

In [ ]:
def create_plot_circulating_amount(df_mint, df_burn):
    token = df_mint.split('/')[1]
    _issue = pd.read_csv(df_mint)
    _destroyedblackfunds = pd.read_csv(df_burn)
    if type(_issue['txvalue'][0])==type(str()):
        _issue['txvalue'] = _issue['txvalue'].astype(float)
        _destroyedblackfunds['txvalue'] = _destroyedblackfunds['txvalue'].astype(float)
    dbf = _destroyedblackfunds.loc[:, ['timestamp', 'txvalue']]
    iss = _issue.loc[:, ['timestamp', 'txvalue']]
    dbf['txvalue'] = dbf['txvalue']*-1
    dfis = pd.concat([dbf,iss])
    dfis = dfis.sort_values('timestamp', axis = 0).reset_index().loc[:,['timestamp', 'txvalue']]
    dfis['utc'] = dfis['timestamp'].apply(lambda x: str(datetime.utcfromtimestamp(x))[0:10])
    dfis = dfis[['utc', 'txvalue']]
    dfis = dfis.groupby('utc').sum()
    a = dfis.index[0]
    b = dfis.index[-1]
    idx = pd.date_range(a,b)
    dfis.index = pd.DatetimeIndex(dfis.index)
    cirulating_amount = dfis.reindex(idx, fill_value=0)
    cirulating_amount.to_csv('plots/{}/{}_circulating_supply.csv'.format(token, token))
    return cirulating_amount

create_plot_circulating_amount(tether_issue, tether_destroyedblackfunds)
create_plot_circulating_amount(usdc_mint, usdc_burn)
create_plot_circulating_amount(paxos_mint, paxos_burn)
create_plot_circulating_amount(dai_mint, dai_burn)
create_plot_circulating_amount(trueusd_mint, trueusd_burn)
create_plot_circulating_amount(husd_mint, husd_burn)
create_plot_circulating_amount(binanceusd_mint, binanceusd_burn)

<center></center>

<center></center>

# Cumulated Balances

In [ ]:
cumsum = pd.Series()
def create_cum_sum(df, st):
    global cumsum
    cs = df.cumsum() + st
    end = cs.iloc[-1]
    cumsum = cumsum.append(cs)
    return end

def create_cum_bal(df):
    global cumsum
    start = 0
    token = df.split("/")[1]
    for i in pd.read_csv(df, chunksize = 1000000):
        i = i['txvalue']
        i = i[i>0]
        if len(i) > 0:
            start = create_cum_sum(i, st = start)
    y = cumsum/cumsum.iloc[-1] #Supply 01 July 20
    x = (np.arange(start = 0 , stop = len(cumsum), step = 1)/len(cumsum))*100
    df = pd.read_csv('plots/{}/{}_balances.csv'.format(token, token))
    df = df.rename(columns={'Unnamed: 0': 'address', 'txvalue': 'balance'})
    df = df[df['balance']>0]
    df = df.reset_index()[['address', 'balance']]
    df['cum'] = cumsum.reset_index()[0]
    df.to_csv('plots/{}/{}_positive_cumulated_balances.csv'.format(token, token))
    cumsum = pd.Series()

create_cum_bal('plots/tether/tether_balances.csv')
create_cum_bal('plots/usdc/usdc_balances.csv')
create_cum_bal('plots/paxos/paxos_balances.csv')
create_cum_bal('plots/dai/dai_balances.csv')
create_cum_bal('plots/binanceusd/binanceusd_balances.csv')
create_cum_bal('plots/husd/husd_balances.csv')
create_cum_bal('plots/trueusd/trueusd_balances.csv')
create_cum_bal('plots/sai/sai_balances.csv')

In [ ]:
#Tether fix
df2 = pd.read_csv('plots/tether/tether_positive_cumulated_balances.csv', index_col=0)
df2.iloc[1:].reset_index().loc[:, 'address':'cum'].to_csv('plots/tether/tether_positive_cumulated_balances.csv')


<center></center>

## Global Transfer count over whole ecosystem

In [ ]:
fr_tether = pd.read_csv(tether_tx_count_from, index_col='Unnamed: 0')
to_tether = pd.read_csv(tether_tx_count_to, index_col='Unnamed: 0')
fr_dai = pd.read_csv(dai_tx_count_from, index_col='Unnamed: 0')
to_dai = pd.read_csv(dai_tx_count_to, index_col='Unnamed: 0')
fr_usdc = pd.read_csv(usdc_tx_count_from, index_col='Unnamed: 0')
to_usdc = pd.read_csv(usdc_tx_count_to, index_col='Unnamed: 0')
fr_paxos = pd.read_csv(paxos_tx_count_from, index_col='Unnamed: 0')
to_paxos = pd.read_csv(paxos_tx_count_to, index_col='Unnamed: 0')
fr_trueusd = pd.read_csv(trueusd_tx_count_from, index_col='Unnamed: 0')
to_trueusd = pd.read_csv(trueusd_tx_count_to, index_col='Unnamed: 0')
fr_binanceusd = pd.read_csv(binanceusd_tx_count_from, index_col='Unnamed: 0')
to_binanceusd = pd.read_csv(binanceusd_tx_count_to, index_col='Unnamed: 0')
fr_husd = pd.read_csv(husd_tx_count_from, index_col='Unnamed: 0')
to_husd = pd.read_csv(husd_tx_count_to, index_col='Unnamed: 0')

fr_new = fr_tether.append([fr_dai, fr_usdc, fr_paxos, fr_trueusd, fr_binanceusd, fr_husd])
fr_new = fr_new.groupby(fr_new.index)['txs'].sum()
fr_new.to_csv('plots/summary/from.csv')

to_new = to_tether.append([to_dai, to_usdc, to_paxos, to_trueusd, to_binanceusd, to_husd])
to_new = to_new.groupby(to_new.index)['txs'].sum()
to_new.to_csv('plots/summary/to.csv')